<a href="https://colab.research.google.com/github/MadMeister/PREN_AVehicle_Plant_id/blob/main/yolo_conversion_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert Yolov3 to Blob


## 1. Setting up Darknet

Note that we are cloning the darknet from [AlexeyAB/darknet](https://github.com/AlexeyAB/darknet.git) which is a fork from official [pjreddie/darknet](https://github.com/pjreddie/darknet) repository. It contains the code for multiple YoloV3 and YoloV4 modifications and is actively maintained.

In [ ]:
%%capture
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
%%capture
!git clone https://github.com/yushulx/opencv-yolo-qr-detection.git

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make
!chmod +x ./darknet

In [ ]:
%cd opencv-yolo-qr-detection
%ls


[Errno 2] No such file or directory: 'opencv-yolo-qr-detection'
/content/darknet
3rdparty/               darknet.py              net_cam_v4.sh*
backup/                 darknet_video.py        obj/
build/                  data/                   README.md
build.ps1*              image_yolov3.sh*        results/
cfg/                    image_yolov4.sh*        scripts/
cmake/                  include/                src/
CMakeLists.txt          json_mjpeg_streams.sh*  vcpkg.json
darknet*                LICENSE                 video_yolov3.sh*
DarknetConfig.cmake.in  Makefile                video_yolov4.sh*
darknet_images.py       net_cam_v3.sh*


### Choosing Tensorflow version

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

'1.15.2'

## 2. Conversion

In the following steps we will show the conversion process of our Yolo model, first to a *.pb* format, and then to OpenVINO IR, from which it will be possible to generate a *.blob* that can run on our OAK devices.

In [ ]:
%cd /content/

/content


### 2.1 External library

We will be using an external library for the conversion.

In [ ]:
!git clone https://github.com/luxonis/yolo2openvino

fatal: destination path 'yolo2openvino' already exists and is not an empty directory.


### 2.2 Convert to .pb

In [ ]:
%cd /content/yolo2openvino

/content/yolo2openvino


In [ ]:
!pip install tensorflow==1.14

In [ ]:
output_name = 'qrcode-yolov3-tiny'
output_name_pb = output_name+'.pb'

In [ ]:
!python3 convert_weights_pb.py \
--yolo 3 \
--weights_file /content/opencv-yolo-qr-detection/qrcode-yolov3-tiny.weights \
--class_names /content/opencv-yolo-qr-detection/qrcode.names \
--output {output_name_pb} \
--tiny \
-h 416 \
-w 416

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

### 2.3 Install OpenVINO 2021.3

In [ ]:
%cd /content/
import os
from urllib.parse import urlparse

## install tools. Open Vino takes some time to download - it's ~400MB
!sudo apt-get install -y pciutils cpio
!sudo apt autoremove

## downnload installation files
url = "https://registrationcenter-download.intel.com/akdlm/irc_nas/17662/l_openvino_toolkit_p_2021.3.394.tgz"
!wget {url}

## Get the name of the tgz
parsed = urlparse(url)
openvino_tgz = os.path.basename(parsed.path)
openvino_folder = os.path.splitext(openvino_tgz)[0]

## Extract & install openvino
!tar xf {openvino_tgz}
%cd {openvino_folder}
!./install_openvino_dependencies.sh && \
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpci3
Suggested packages:
  libarchive1
The following NEW packages will be installed:
  cpio libpci3 pciutils
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 368 kB of archives.
After this operation, 1,786 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 cpio amd64 2.12+dfsg-6ubuntu0.18.04.4 [86.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpci3 amd64 1:3.5.2-1ubuntu1.1 [24.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 pciutils amd64 1:3.5.2-1ubuntu1.1 [257 kB]
Fetched 368 kB in 6s (58.3 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76

### 2.4 Setting up the JSON file

In [ ]:
%cd /content/

/content


We copy the JSON file required for the conversion from the yolo2openvino/json directory. 

In [ ]:
yolo_version=3

In [ ]:
if yolo_version == 3:
  !cp /content/yolo2openvino/json/yolo_v3_tiny.json /content/yolo.json
else:
  !cp /content/yolo2openvino/json/yolo_v4_tiny_custom.json /content/yolo.json

<font color="red">**IMPORTANT: You have to correctly set up the contents of this file**</font>. If you have not changed anchors and masks, then you only have to change the `classes`. Make sure the following are correct:

* *id* should be left as it is whether we used YoloV3 or YoloV4 version,
* *classes* must match the number of the classes your trained Yolo can detect,
* *anchors* must be changed if you used different anchors,
* we leave *coords* as they are,
* *num* must be set to *6* if you used default YoloV3-tiny or YoloV4-tiny configuration,
* *mask* must be set to `[[3, 4, 5], [1, 2, 3]]` for YoloV4-tiny and `[[3, 4, 5], [0, 1, 2]]` **for YoloV3-tiny** and **for YoloV4-tiny-custom** (you can find this in the *.cfg* file under [yolo] layers - if you have not changed the code, by default YoloV3-tiny and YoloV4-tiny-custom configs are used in this tutorial),
* *entry points* must be 

 `["detector/yolo-v4-tiny/Reshape", "detector/yolo-v4-tiny/Reshape_4"]` for YoloV4-tiny, and 

 `["detector/yolo-v3-tiny/Reshape", "detector/yolo-v3-tiny/Reshape_4"]` for YoloV3-tiny!

These should be set automatically in the next cell, but please ensure the values are correct.

In [ ]:
labels = ['QR_CODE']

In [ ]:
import re
tiny_yolo_json = '/content/yolo.json'
with open(tiny_yolo_json) as f:
    s = f.read()
s = re.sub('"classes": \d*','"classes": ' + str(len(labels)),s)
with open(tiny_yolo_json, 'w') as f:
  f.write(s)
#Check that the number of classes is correct (same as the number of your training labels)|
!cat /content/yolo.json 

[
  {
    "id": "TFYOLOV3",
    "match_kind": "general",
    "custom_attributes": {
      "classes": 1,
      "anchors": [10, 14, 23, 27, 37, 58, 81, 82, 135, 169, 344, 319],
      "coords": 4,
      "num": 6,
      "masks": [[3, 4, 5], [0, 1, 2]],
      "entry_points": ["detector/yolo-v3-tiny/Reshape", "detector/yolo-v3-tiny/Reshape_4"]
    }
  }
]

### 2.5 Conversion to OpenVINO IR

We can now proceed with the conversion to OpenVINO IR.

In [ ]:
input_model = f"/content/yolo2openvino/qrcode-yolov3-tiny.pb"
input_model

'/content/yolo2openvino/qrcode-yolov3-tiny.pb'

In [ ]:
# Get openvino installation path
openvino = !find /opt/intel -type d -name openvino*
!python -mpip install -r {openvino[0]}/deployment_tools/model_optimizer/requirements.txt

Ignoring tensorflow: markers 'python_version >= "3.8"' don't match your environment


In [ ]:
!python {openvino[0]}/deployment_tools/model_optimizer/mo.py \
--input_model $input_model \
--tensorflow_use_custom_operations_config /content/yolo.json \
--batch 1 \
--data_type FP16 \
--reverse_input_channel \
--model_name qrcode-yolov3-tiny \
--output_dir /content/output/

[ WARNING ]  Use of deprecated cli option --tensorflow_use_custom_operations_config detected. Option use in the following releases will be fatal. Please use --transformations_config cli option instead
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/content/yolo2openvino/qrcode-yolov3-tiny.pb
	- Path for generated IR: 	/content/output/
	- IR output name: 	qrcode-yolov3-tiny
	- Log level: 	ERROR
	- Batch: 	1
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	True
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoar

### 2.6 Use blobconverter to convert to blob that can be run on OAK devices

Set up the paths.

In [ ]:
# Choose the directory where you would like to save the blob.
blob_dir = "/content/output/"

binfile = f"/content/output/{output_name}.bin"
xmlfile = f"/content/output/{output_name}.xml"

Install latest version of blobconverter.

In [ ]:
!python -m pip install blobconverter==1.2.7

     |████████████████████████████████| 131 kB 4.4 MB/s 
     |████████████████████████████████| 7.7 MB 35.1 MB/s 
     |████████████████████████████████| 73 kB 1.9 MB/s 


Convert the model to blob and download the file.

In [ ]:
import blobconverter
blob_path = blobconverter.from_openvino(
    xml=xmlfile,
    bin=binfile,
    data_type="FP16",
    shaves=10,
    version="2021.3",
    use_cache=False
)
from google.colab import files
files.download(blob_path)

[==================================================]
Done


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download the OpenVINO IR files.

In [ ]:
# Name the archive as you see fit
!tar czvf yolo_tiny_car.tar.gz /content/output
# Save the archive to the gdrive
from google.colab import files
files.download("yolo_tiny_car.tar.gz")

tar: Removing leading `/' from member names
/content/output/
/content/output/qrcode-yolov3-tiny.xml
/content/output/qrcode-yolov3-tiny.mapping
/content/output/qrcode-yolov3-tiny.bin


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Next steps

For more information on next steps, you can check out our [documentation](https://docs.luxonis.com/projects/api/en/latest/samples/Yolo/tiny_yolo/).

* setNumClasses - number of YOLO classes *(1 in our case)*, 
* setCoordinateSize - size of coordinates *(4 by default)*,
* setAnchors - yolo anchors *(located in [yolo] layer in CFG)*,
* setIouThreshold - intersection over union threshold,
* setConfidenceThreshold - confidence threshold above which objects are detected,
* setAnchorMasks - set the anchor masks as desribed in the next section.

#### Anchor masks

There are 2 (or 3 for non-tiny Yolos) [yolo] layers in Yolo architecture, which you can also find in the config. The first corresponds to the larger objects, while the second corresponds to the smaller objects. This corresponds with masks, which tell model which anchors to use. Mask *3,4,5* will tell the model to use 4th, 5th, and 6th anchor for example. You will find layers for bigger objects higher in the CFG.

When setting up the masks in the *YoloDetectionNetwork*, you also have to change the number in `side`. You have to change it as follows:
`side32` to `sideX` and `side16` to `sideY`, where `X = width/16` and `Y = width/32`.

If you are using non-tiny Yolo model, then you should have 3 [yolo] layers, and you should have `sideX`, `sideY`, and `sideZ`, where `X = width/8` and `Y = width/16`, and `Z = width/32`.
